<a href="https://colab.research.google.com/github/gowtham-dd/Fish-Classification-using-Deep-Learning/blob/main/Fish_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
crowww_a_large_scale_fish_dataset_path = kagglehub.dataset_download('crowww/a-large-scale-fish-dataset')

print('Data source import complete.')


## IMPORTS

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

from sklearn.model_selection import train_test_split

import tensorflow as tf

2025-05-09 06:20:00.660853: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746771600.902651      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746771600.972649      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
image_dir = Path('../input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset')

## Creating DF

In [ ]:
# Get filepaths and labels
filepaths = list(image_dir.glob(r'**/*.png'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

# Concatenate filepaths and labels
image_df = pd.concat([filepaths, labels], axis=1)

# Drop GT images
image_df['Label'] = image_df['Label'].apply(lambda x: np.NaN if x[-2:] == 'GT' else x)
image_df = image_df.dropna(axis=0)

# Sample 200 images from each class
samples = []

for category in image_df['Label'].unique():
    category_slice = image_df.query("Label == @category")
    samples.append(category_slice.sample(200, random_state=1))

image_df = pd.concat(samples, axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)

In [ ]:
image_df

In [ ]:
train_df, test_df = train_test_split(image_df, train_size=0.7, shuffle=True, random_state=1)

## Loading Images

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

## Load Pre Trained Model

In [ ]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrained_model.trainable = False

## Training

In [ ]:
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(9, activation='softmax')(x)


model = tf.keras.Model(inputs=inputs, outputs=outputs)


model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

In [ ]:
results = model.evaluate(test_images, verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

## All Models

In [ ]:
import os
import numpy as np
import pandas as pd
from pathlib import Path

# Define your image directory
image_dir = Path('../input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset') # <--- change this to your dataset path

# Get filepaths and labels
filepaths = list(image_dir.glob(r'**/*.png'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

# Concatenate filepaths and labels
image_df = pd.concat([filepaths, labels], axis=1)

# Drop GT images
image_df['Label'] = image_df['Label'].apply(lambda x: np.NaN if x[-2:] == 'GT' else x)
image_df = image_df.dropna(axis=0)

# ✅ Sample up to 200 images per class — skip classes with fewer images
samples = []
min_required = 200  # number of samples per class

for category in image_df['Label'].unique():
    category_slice = image_df.query("Label == @category")
    if len(category_slice) >= min_required:
        samples.append(category_slice.sample(min_required, random_state=1))
    else:
        print(f"Skipping class '{category}' — only {len(category_slice)} images available.")

# ✅ Check if we have valid data
if not samples:
    raise ValueError("❌ No categories had enough images to include in the dataset.")

# ✅ Final dataset
image_df = pd.concat(samples, axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import os

from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV2, InceptionV3, EfficientNetB0

# Assume image_df is already loaded and contains 'Filepath' and 'Label' columns
train_df, val_df = train_test_split(image_df, test_size=0.2, random_state=42)

# Use MobileNetV2 preprocessing by default (works with all as input normalized between -1 and 1)
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)
val_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
)

val_images = val_generator.flow_from_dataframe(
    dataframe=val_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False,
)

# Helper to create model on top of base model
def create_model(base_model, num_classes):
    base_model.trainable = False
    x = tf.keras.layers.Dense(128, activation='relu')(base_model.output)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs=base_model.input, outputs=outputs)

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Store results
results = []


Found 1440 validated image filenames belonging to 9 classes.
Found 360 validated image filenames belonging to 9 classes.


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd

# Make sure these are already defined earlier:
# - create_model function
# - train_images and val_images (ImageDataGenerators)
# - results list

# Step 1: Load VGG16 base model
print("Training VGG16...")
vgg = VGG16(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

# Step 2: Create your model using your custom function
model_vgg = create_model(vgg, num_classes=len(train_images.class_indices))

# Step 3: Train the model
history_vgg = model_vgg.fit(
    train_images,
    validation_data=val_images,
    epochs=10,
    callbacks=[
        EarlyStopping(patience=3, monitor='val_loss', restore_best_weights=True)
    ],
    verbose=1
)

# Step 4: Evaluate on validation data
val_loss, val_acc = model_vgg.evaluate(val_images)

# Step 5: Append the result
results = []  # Ensure this list exists
results.append({"Model": "VGG16", "Val Accuracy": val_acc})

# Step 6: Save the trained model
model_vgg.save("VGG16.h5")
print("VGG16 model saved as VGG16.h5")


Training VGG16...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


45/45 ━━━━━━━━━━━━━━━━━━━━ 545s 12s/step - accuracy: 0.2089 - loss: 2.1201 - val_accuracy: 0.5056 - val_loss: 1.6314
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 536s 12s/step - accuracy: 0.6522 - loss: 1.4320 - val_accuracy: 0.7222 - val_loss: 1.0766
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 536s 12s/step - accuracy: 0.7837 - loss: 0.8776 - val_accuracy: 0.8083 - val_loss: 0.6937
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 536s 12s/step - accuracy: 0.8635 - loss: 0.5646 - val_accuracy: 0.8917 - val_loss: 0.4281
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 536s 12s/step - accuracy: 0.9076 - loss: 0.3805 - val_accuracy: 0.9389 - val_loss: 0.3111
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 537s 12s/step - accuracy: 0.9505 - loss: 0.2610 - val_accuracy: 0.9417 - val_loss: 0.2462
Epoch 7/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 535s 12s/step - accuracy: 0.9655 - loss: 0.2066 - val_accuracy: 0.9556 - val_loss: 0.2338
Epoch 8/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 574s 13s/step - accuracy: 0.9621 - loss: 0.1818 - val_accuracy: 0.9556 - val_

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping

# Make sure these are already defined:
# - create_model() function
# - train_images and val_images (ImageDataGenerator)
# - results list (should already exist or be initialized before this block)

print("Training ResNet50...")

# Step 1: Load the ResNet50 base model
resnet = ResNet50(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

# Step 2: Create your full model
model_resnet = create_model(resnet, num_classes=len(train_images.class_indices))
# etc.

# Step 3: Train the model
history_resnet = model_resnet.fit(
    train_images,
    validation_data=val_images,
    epochs=10,
    callbacks=[
        EarlyStopping(patience=3, monitor='val_loss', restore_best_weights=True)
    ],
    verbose=1
)

# Step 4: Evaluate the model
val_loss, val_acc = model_resnet.evaluate(val_images)

# Step 5: Append the result to results list
results.append({"Model": "ResNet50", "Val Accuracy": val_acc})

# Step 6: Save the model
model_resnet.save("ResNet50.h5")
print("ResNet50 model saved as ResNet50.h5")


Training ResNet50...
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 200s 4s/step - accuracy: 0.1880 - loss: 2.1055 - val_accuracy: 0.4389 - val_loss: 1.7135
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 181s 4s/step - accuracy: 0.3889 - loss: 1.7105 - val_accuracy: 0.4778 - val_loss: 1.4955
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 188s 4s/step - accuracy: 0.4871 - loss: 1.4959 - val_accuracy: 0.5528 - val_loss: 1.3089
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 189s 4s/step - accuracy: 0.5304 - loss: 1.3253 - val_accuracy: 0.5389 - val_loss: 1.2003
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 187s 4s/step - accuracy: 0.5712 - loss: 1.2508 - val_accuracy: 0.5667 - val_loss: 1.1693
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 182s 4s/step - accuracy: 0.6150 - loss: 1.1258 - val_accuracy: 0.5444 - val_loss: 1.2349
Epoch 7/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 187s 4s/step - accuracy: 0.6159 - loss: 1.1315 - val_accuracy: 0.6167 - val_loss: 1.0120
Epoch 8/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping

# Assumes:
# - `create_model()` is defined and returns a compiled model.
# - `train_images` and `val_images` are preprocessed and ready ImageDataGenerators.
# - `results` is a list already initialized to store results.

print("Training MobileNetV2...")

# Step 1: Load the MobileNetV2 base model
mobilenet = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

# Step 2: Create your full model
model_mobilenet = create_model(mobilenet, len(train_images.class_indices))

# Step 3: Train the model
history_mobilenet = model_mobilenet.fit(
    train_images,
    validation_data=val_images,
    epochs=10,
    callbacks=[
        EarlyStopping(patience=3, monitor='val_loss', restore_best_weights=True)
    ],
    verbose=1
)

# Step 4: Evaluate the model
val_loss, val_acc = model_mobilenet.evaluate(val_images)

# Step 5: Append the result
results.append({"Model": "MobileNetV2", "Val Accuracy": val_acc})

# Step 6: Save the model
model_mobilenet.save("MobileNetV2.h5")
print("MobileNetV2 model saved as MobileNetV2.h5")


Training MobileNetV2...
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 74s 1s/step - accuracy: 0.6099 - loss: 1.1726 - val_accuracy: 0.9556 - val_loss: 0.1253
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 61s 1s/step - accuracy: 0.9846 - loss: 0.0659 - val_accuracy: 0.9806 - val_loss: 0.0490
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 61s 1s/step - accuracy: 0.9971 - loss: 0.0232 - val_accuracy: 0.9917 - val_loss: 0.0367
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 1.0000 - loss: 0.0089 - val_accuracy: 0.9917 - val_loss: 0.0269
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 1.0000 - loss: 0.0040 - val_accuracy: 0.9861 - val_loss: 0.0275
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 61s 1s/step - accuracy: 1.0000 - loss: 0.0027 - val_accuracy: 0.9889 - val_loss: 0.0214
Epoch 7/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 61s 1s/step - accuracy: 1.0000 - loss: 0.0020 - val_accuracy: 0.9944 - val_loss: 0.0193
Epoch 8/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 61s 1s/

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.callbacks import EarlyStopping

# Assumes:
# - create_model() is defined and returns a compiled model.
# - train_images and val_images are prepared ImageDataGenerators.
# - results is a list already initialized to store results.

print("Training InceptionV3...")

# Step 1: Load the InceptionV3 base model
inception = InceptionV3(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

# Step 2: Create your full model
model_inception = create_model(inception, num_classes=len(train_images.class_indices))

# Step 3: Train the model
history_inception = model_inception.fit(
    train_images,
    validation_data=val_images,
    epochs=10,
    callbacks=[
        EarlyStopping(patience=3, monitor='val_loss', restore_best_weights=True)
    ],
    verbose=1
)

# Step 4: Evaluate the model
val_loss, val_acc = model_inception.evaluate(val_images)

# Step 5: Append the result
results.append({"Model": "InceptionV3", "Val Accuracy": val_acc})

# Step 6: Save the model
model_inception.save("InceptionV3.h5")
print("InceptionV3 model saved as InceptionV3.h5")


Training InceptionV3...
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 158s 3s/step - accuracy: 0.5668 - loss: 1.3147 - val_accuracy: 0.8972 - val_loss: 0.2966
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 120s 3s/step - accuracy: 0.9333 - loss: 0.2224 - val_accuracy: 0.9389 - val_loss: 0.1605
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 142s 3s/step - accuracy: 0.9732 - loss: 0.0970 - val_accuracy: 0.9556 - val_loss: 0.1678
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 119s 3s/step - accuracy: 0.9849 - loss: 0.0603 - val_accuracy: 0.9389 - val_loss: 0.1619
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 142s 3s/step - accuracy: 0.9774 - loss: 0.0631 - val_accuracy: 0.9667 - val_loss: 0.1134
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 120s 3s/step - accuracy: 0.9936 - loss: 0.0308 - val_accuracy: 0.9722 - val_loss: 0.0777
Epoch 7/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 119s 3s/step - accuracy: 1.0000 - loss: 0.0112 - val_accuracy: 0.9694 - val_loss: 0.0767
Epoch 8/10
45/45 ━━━━━━━━━━━━━━━━━━━

In [ ]:
results_df = pd.DataFrame(results)
print(results_df)

         Model  Val Accuracy
0        VGG16      0.966667
1     ResNet50      0.702778
2  MobileNetV2      0.997222
3  InceptionV3      0.975000


## Model Saving

In [ ]:
# Assuming you still have each trained model in memory:
# Save each one manually like this:

model_vgg.save("VGG16.h5")
model_resnet.save("ResNet50_v3.h5")
model_mobilenet.save("MobileNetV2.h5")
model_inception.save("InceptionV3.h5")


## Streamlit

In [ ]:
pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 4.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 6.8 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import streamlit as st
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import json
import os

st.title("🐟 Fish Classifier App")
st.write("Upload a fish image and select a model to predict its class with confidence scores.")

# Available models and their preprocessing
model_files = {
    "VGG16": ("VGG16.h5", tf.keras.applications.vgg16.preprocess_input),
    "ResNet50 v3": ("ResNet50_v3.h5", tf.keras.applications.resnet50.preprocess_input),
    "MobileNetV2": ("MobileNetV2.h5", tf.keras.applications.mobilenet_v2.preprocess_input),
    "InceptionV3": ("InceptionV3.h5", tf.keras.applications.inception_v3.preprocess_input),
}

model_choice = st.selectbox("Choose a model", list(model_files.keys()))
uploaded_image = st.file_uploader("Upload a fish image", type=['jpg', 'jpeg', 'png'])

# Load label mapping
@st.cache_data
def load_class_names():
    try:
        with open("class_indices.json", "r") as f:
            class_indices = json.load(f)
        # Reverse map: index -> class name
        return {v: k for k, v in class_indices.items()}
    except:
        return {i: f"Class {i}" for i in range(9)}  # fallback for 9 classes

# Load model
@st.cache_resource
def load_model(model_path):
    return tf.keras.models.load_model(model_path)

# Predict
if uploaded_image is not None and model_choice:
    model_path, preprocess_fn = model_files[model_choice]
    model = load_model(model_path)
    class_names = load_class_names()

    image = Image.open(uploaded_image).convert('RGB')
    image_resized = image.resize((224, 224))
    image_array = tf.keras.preprocessing.image.img_to_array(image_resized)
    image_array = preprocess_fn(image_array)
    image_array = np.expand_dims(image_array, axis=0)

    predictions = model.predict(image_array)[0]
    predicted_index = np.argmax(predictions)
    confidence = predictions[predicted_index]
    predicted_label = class_names.get(predicted_index, f"Class {predicted_index}")

    # Show results
    st.image(image, caption="Uploaded Image", use_column_width=True)
    st.subheader(f"Predicted Class: 🐠 {predicted_label}")
    st.write(f"Confidence: **{confidence * 100:.2f}%**")

    # Plot confidence scores
    st.subheader("📊 Confidence Scores")
    fig, ax = plt.subplots()
    bars = ax.bar(range(len(predictions)), predictions, tick_label=[class_names.get(i, f"C{i}") for i in range(len(predictions))])
    ax.set_xlabel("Class")
    ax.set_ylabel("Confidence")
    ax.set_title("Confidence for Each Class")

    for bar in bars:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2.0, yval + 0.01, f"{yval:.2f}", ha='center', va='bottom', fontsize=8)

    st.pyplot(fig)


2025-05-09 17:51:40.673 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 17:51:40.676 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 17:51:40.678 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 17:51:40.679 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 17:51:40.680 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 17:51:40.681 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 17:51:40.682 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 17:51:40.683 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

## Custom CNN

In [ ]:
train_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_generator = ImageDataGenerator(rescale=1./255)


NameError: name 'ImageDataGenerator' is not defined

In [ ]:

from tensorflow.keras import layers, models, optimizers, callbacks
import tensorflow as tf

# Define Custom CNN model
def create_custom_cnn(input_shape=(224, 224, 3), num_classes=9):
    model = models.Sequential()

    # Conv Block 1
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    # Conv Block 2
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Conv Block 3
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Fully Connected Layers
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))

    # Compile
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Get number of classes dynamically from training data
num_classes = train_images.num_classes

# Create and train model
model_cnn = create_custom_cnn(input_shape=(224, 224, 3), num_classes=num_classes)

# Train
history = model_cnn.fit(
    train_images,
    validation_data=val_images,
    epochs=10,
    callbacks=[callbacks.EarlyStopping(patience=3, monitor='val_loss', restore_best_weights=True)],
    verbose=1
)

# Evaluate on test set
test_loss, test_acc = model_cnn.evaluate(test_images)
print(f"Test Accuracy: {test_acc:.4f}")

# Save model
model_cnn.save("Custom_CNN_FishClassifier.h5")


AttributeError: 'DataFrameIterator' object has no attribute 'num_classes'